In [1]:
import pandas as pd
import numpy as np

# === 1. File settings ===
file_path = 'S000013_002.merp'
output_file = 'S10_evening_OD.xlsx'

# === 2. Data loading and preprocessing ===
#df = pd.read_csv(file_path, sep=';', header=0, low_memory=False)
# Find and load data from the header row
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

header_line_idx = None
for i, line in enumerate(lines):
    if 'MeasurementAreaNo' in line and 'PedNo' in line:
        header_line_idx = i
        break

if header_line_idx is None:
    raise ValueError("❌ Could not find the header line in the input file.")

df = pd.read_csv(file_path, sep=';', header=0, skiprows=header_line_idx, low_memory=False)

df.columns = df.columns.str.strip()
df_filtered = df[['MeasurementAreaNo', 'tEnter', 'PedNo']].copy()
df_filtered = df_filtered.astype(str).apply(lambda col: col.str.strip())
df_filtered = df_filtered[
    df_filtered['MeasurementAreaNo'].str.isnumeric() &
    df_filtered['tEnter'].str.replace('.', '', 1).str.isnumeric() &
    df_filtered['PedNo'].str.isnumeric()
]
df_filtered['MeasurementAreaNo'] = df_filtered['MeasurementAreaNo'].astype(int)
df_filtered['tEnter'] = df_filtered['tEnter'].astype(float)
df_filtered['PedNo'] = df_filtered['PedNo'].astype(int)

# === 3. Identify entry and exit zones for each pedestrian ===
start_zones = df_filtered.loc[df_filtered.groupby('PedNo')['tEnter'].idxmin()]
end_zones = df_filtered.loc[df_filtered.groupby('PedNo')['tEnter'].idxmax()]
trajectories_all = pd.merge(
    start_zones[['PedNo', 'MeasurementAreaNo', 'tEnter']],
    end_zones[['PedNo', 'MeasurementAreaNo']],
    on='PedNo',
    suffixes=('_start', '_end')
)

# Exclude zones 12 and 13
all_zones = sorted(set(trajectories_all['MeasurementAreaNo_start']) | set(trajectories_all['MeasurementAreaNo_end']))
zones_to_use = [z for z in all_zones if z not in [12, 13]]

# === 4. Build OD matrix ===
def build_matrix(df, zones):
    matrix = df.pivot_table(
        index='MeasurementAreaNo_start',
        columns='MeasurementAreaNo_end',
        aggfunc='size',
        fill_value=0
    )
    return matrix.reindex(index=zones, columns=zones, fill_value=0)

# === 5. Transformation steps ===
def apply_steps_5_and_6(matrix):
    m = matrix.copy().astype(float)
    row_labels = m.index.tolist()
    col_labels = m.columns.tolist()

    # Step 5: diagonal redistribution
    for diag in range(3, 12):
        if diag in row_labels and diag in col_labels:
            diag_val = m.loc[diag, diag]
            if diag_val > 0:
                col_vals = m.loc[:, diag].drop(index=diag)
                col_sum = col_vals.sum()
                if col_sum > 0:
                    for r in col_vals.index:
                        m.loc[r, diag] += diag_val * (m.loc[r, diag] / col_sum)
                m.loc[diag, diag] = 0

    # Additional redistribution for specific cells
    for r, c in [(3, 4), (4, 3), (10, 11), (11, 10)]:
        if r in row_labels and c in col_labels:
            val = m.loc[r, c]
            if val > 0:
                row_vals = m.loc[r, :].drop(labels=c)
                row_sum = row_vals.sum()
                if row_sum > 0:
                    for cc in row_vals.index:
                        m.loc[r, cc] += val * (m.loc[r, cc] / row_sum)
                m.loc[r, c] = 0

    return m.round().astype(int)

def apply_all_steps(matrix):
    m = matrix.copy().astype(float)

    def get(r, c): return m.loc[r, c]
    def set_(r, c, val): m.loc[r, c] = val
    def add(r, c, val): m.loc[r, c] += val

    for c in [7, 8, 9, 11]:
        if 5 in m.index and c in m.columns:
            add(8, c, get(5, c)); set_(5, c, 0)
    for c in [3, 4, 5, 6, 10]:
        if 8 in m.index and c in m.columns:
            add(5, c, get(8, c)); set_(8, c, 0)
    for r in [3, 4, 5, 6, 10]:
        if r in m.index and 8 in m.columns and 5 in m.columns:
            add(r, 5, get(r, 8)); set_(r, 8, 0)
    for r in [7, 8, 9, 11]:
        if r in m.index and 5 in m.columns and 8 in m.columns:
            add(r, 8, get(r, 5)); set_(r, 5, 0)

    return m.round().astype(int)

# === 6. Write results to Excel ===
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

    def write_matrices(df_traj, sheet_name, add_scaled=False):
        base_matrix = build_matrix(df_traj, zones_to_use)
        matrix_1 = apply_steps_5_and_6(base_matrix)
        matrix_2 = apply_steps_5_and_6(apply_all_steps(base_matrix))

        # First matrix
        matrix_1.to_excel(writer, sheet_name=sheet_name, startrow=0, startcol=0)
        if add_scaled:
            (matrix_1 * 12).to_excel(writer, sheet_name=sheet_name, startrow=0, startcol=len(matrix_1.columns) + 3)

        # Second matrix
        second_start = len(matrix_1) + 4
        matrix_2.to_excel(writer, sheet_name=sheet_name, startrow=second_start, startcol=0)
        if add_scaled:
            (matrix_2 * 12).to_excel(writer, sheet_name=sheet_name, startrow=second_start, startcol=len(matrix_2.columns) + 3)

    # PedestrianTimes sheet
    t_enter = df_filtered.groupby('PedNo')['tEnter'].min()
    t_leave = df_filtered.groupby('PedNo')['tEnter'].max()
    zone_counts = df_filtered[df_filtered['MeasurementAreaNo'].isin([12, 13])]
    zone_counts = zone_counts.groupby(['PedNo', 'MeasurementAreaNo']).size().unstack(fill_value=0)
    zone_counts.columns = zone_counts.columns.map({12: 'N, times', 13: 'S, times'})
    for col in ['N, times', 'S, times']:
        if col not in zone_counts.columns:
            zone_counts[col] = 0

    ped_time_df = pd.DataFrame({
        'Pedestrian ID': t_enter.index,
        'Entry Time (s)': t_enter.values,
        'Exit Time (s)': t_leave.values,
    })
    ped_time_df['Total Time (s)'] = ped_time_df['Exit Time (s)'] - ped_time_df['Entry Time (s)']
    ped_time_df = ped_time_df.merge(zone_counts, how='left', left_on='Pedestrian ID', right_index=True)
    ped_time_df['N, times'] = ped_time_df['N, times'].fillna(0).astype(int)
    ped_time_df['S, times'] = ped_time_df['S, times'].fillna(0).astype(int)
    ped_time_df.to_excel(writer, sheet_name='PedestrianTimes', index=False)

    # Statistics sheet
    stats = {
        'Total Pedestrians': [ped_time_df.shape[0]],
        'Total Time (s)': [ped_time_df['Total Time (s)'].sum()],
        'Average Time (s)': [ped_time_df['Total Time (s)'].mean()],
        'Median Time (s)': [ped_time_df['Total Time (s)'].median()],
        'N tunnel total': [ped_time_df['N, times'].sum()],
        'S tunnel total': [ped_time_df['S, times'].sum()],
    }
    pd.DataFrame(stats).to_excel(writer, sheet_name='Statistics', index=False)

    # All_Trajectories (no scaled versions)
    write_matrices(trajectories_all, "All_Trajectories", add_scaled=False)

    # Interval sheets (with scaled matrices ×12)
    for i in range(12):
        lower = i * 300
        upper = (i + 1) * 300 if i < 11 else float('inf')
        interval_df = trajectories_all[
            (trajectories_all['tEnter'] >= lower) &
            (trajectories_all['tEnter'] < upper)
        ]
        name = f'Interval_{i+1}_{int(lower)}_{"" if upper == float("inf") else int(upper)}'
        write_matrices(interval_df, name[:31], add_scaled=True)

# === 7. Final output to console ===
total_pedestrians = df_filtered['PedNo'].nunique()
print(f"✅ Excel file saved successfully: {output_file}")
print(f"👣 Total number of unique pedestrians: {total_pedestrians}")


✅ Excel file saved successfully: S10_evening_OD.xlsx
👣 Total number of unique pedestrians: 8921
